In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup

In [2]:
def get_html(url):
    try:
        with closing(get(url,stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                if resp.status_code != 200:
                    return 'Error: Status Code {0}'.format(resp.status_code)
                else:
                    return 'Error: Content Type = {0}'.format(resp.headers['Content-Type'])
    except RequestException as e:
        print('Error during requests to {0}: {1}'.format(url,str(e)))
        
def is_good_response(resp):
    content_type = resp.headers['Content-Type'].lower()
    return(resp.status_code == 200 and content_type is not None and content_type.find('html') > -1)

In [3]:
raw_html = get_html('http://www.fabpedigree.com/james/mathmen.htm')
html = BeautifulSoup(raw_html,'lxml')
for i,li in enumerate(html.select('li')):
    print(i,li.text)

0  Isaac Newton

1  Archimedes

2  Carl F. Gauss

3  Leonhard Euler

4  Bernhard Riemann

5  Henri Poincaré

6  Joseph-Louis Lagrange

7  Euclid  of Alexandria

8  David Hilbert

9  Gottfried W. Leibniz

10  Alexandre Grothendieck

11  Pierre de Fermat

12  Évariste Galois

13  John von Neumann

14  René Descartes

15  Karl W. T. Weierstrass

16  Srinivasa Ramanujan

17  Hermann K. H. Weyl

18  Peter G. L. Dirichlet

19  Niels Abel

20  Georg Cantor

21  Carl G. J. Jacobi

22  Brahmagupta

23  Augustin Cauchy

24  Arthur Cayley

25  Emmy Noether

26  Pythagoras  of Samos

27  Aryabhata

28  Leonardo `Fibonacci'

29  William R. Hamilton

30  Apollonius  of Perga

31  Charles Hermite

32  Pierre-Simon Laplace

33  Carl Ludwig Siegel

34  Diophantus  of Alexandria

35  Muhammed al-Khowârizmi

36  Richard Dedekind

37  Kurt Gödel

38  Bháscara (II) Áchárya

39  Felix Christian Klein

40  Blaise Pascal

41  Élie Cartan

42  Archytas  of Tarentum

43  Godfrey H. Hardy

44  Alhazen ibn al-Hay

In [4]:
names = []
for li in html.select('li'):
    names.append(li.text.rstrip())
#clean up 74
names[74] = names[74].split('\n')[0]

In [5]:
def get_hits(name):
    url = 'https://xtools.wmflabs.org/articleinfo/en.wikipedia.org/{0}'.format(name)
    resp = get_html(url)
    html = BeautifulSoup(resp,'lxml')
    for a in html.select('a'):
        if a['href'].find('latest-60') > -1:
            return int(a.text.replace(',',''))

In [6]:
def get_hits_on_name(name):
    """
    Accepts a `name` of a mathematician and returns the number
    of hits that mathematician's Wikipedia page received in the 
    last 60 days, as an `int`
    """
    # url_root is a template string that is used to build a URL.
    url_root = 'https://xtools.wmflabs.org/articleinfo/en.wikipedia.org/{}'
    response = get_html(url_root.format(name))

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')

        hit_link = [a for a in html.select('a')
                    if a['href'].find('latest-60') > -1]

        if len(hit_link) > 0:
            # Strip commas
            link_text = hit_link[0].text.replace(',', '')
            try:
                # Convert to integer
                return int(link_text)
            except:
                log_error("couldn't parse {} as an `int`".format(link_text))

    log_error('No pageviews found for {}'.format(name))
    return None

In [7]:
result = []
for i,name in enumerate(names):
    result.append((name,get_hits(name)))
    print(int((i/len(names)*100)),'%', end='\r')
print(result)

incomplete = []
for i in result:
    if i[1] == None:
        incomplete.append(i)

[(' Isaac Newton', 565211), (' Archimedes', 165054), (' Carl F. Gauss', 52), (' Leonhard Euler', 99447), (' Bernhard Riemann', 29125), (' Henri Poincaré', 40044), (' Joseph-Louis Lagrange', 25869), (' Euclid  of Alexandria', 362), (' David Hilbert', 40608), (' Gottfried W. Leibniz', None), (' Alexandre Grothendieck', 141), (' Pierre de Fermat', 23487), (' Évariste Galois', 33842), (' John von Neumann', 128040), (' René Descartes', 214691), (' Karl W. T. Weierstrass', None), (' Srinivasa Ramanujan', 385192), (' Hermann K. H. Weyl', None), (' Peter G. L. Dirichlet', None), (' Niels Abel', 171), (' Georg Cantor', 43792), (' Carl G. J. Jacobi', 10), (' Brahmagupta', 38452), (' Augustin Cauchy', 3214), (' Arthur Cayley', 5805), (' Emmy Noether', 31073), (' Pythagoras  of Samos', 333), (' Aryabhata', 176654), (" Leonardo `Fibonacci'", None), (' William R. Hamilton', None), (' Apollonius  of Perga', 8302), (' Charles Hermite', 3997), (' Pierre-Simon Laplace', 38868), (' Carl Ludwig Siegel', 1

In [8]:
print(incomplete)

[(' Gottfried W. Leibniz', None), (' Karl W. T. Weierstrass', None), (' Hermann K. H. Weyl', None), (' Peter G. L. Dirichlet', None), (" Leonardo `Fibonacci'", None), (' William R. Hamilton', None), (' Muhammed al-Khowârizmi', None), (' Bháscara (II) Áchárya', None), (' Alhazen ibn al-Haytham', None), (' F.E.J. Émile Borel', None), (' F. Gotthold Eisenstein', None), (' Panini  of Shalatula', None), (' Hermann G. Grassmann', None), (' M. E. Camille Jordan', None), (' F. L. Gottlob Frege', None), (' James J. Sylvester', None), (' Ernst E. Kummer', None), (' Omar al-Khayyám', None), (' Adrien M. Legendre', None)]


In [9]:
# Manually Fixed names
fixed_list = ['Gottfried Wilhelm Leibniz', 'Karl Weierstrass', 'Hermann Weyl', 'Peter Gustav Lejeune Dirichlet', 
         'Fibonacci', 'William Rowan Hamilton', 'Muhammad_ibn_Musa_al-Khwarizmi', 'Émile Borel', 'Gotthold Eisenstein',
         'Pāṇini','Hermann Grassmann', 'Camille Jordan', 'Gottlob Frege','James Joseph Sylvester', 'Ernst Kummer', 
         'Omar Khayyám','Adrien-Marie Legendre']

In [10]:
for i,name in enumerate(fixed_list):
    result.append((name,get_hits(name)))
    print(int((i/len(fixed_list)*100)),'%', end='\r')

print(len(result))

101%


In [17]:
for i in result:
    if i[1] == None:
        result.remove(i)

In [18]:
def take_second(elem):
    return elem[1]
final_result = sorted(result,reverse=True, key=take_second)
for i in range(0,10):
    print(i+1,final_result[i][0],':  ',final_result[i][1])

1  Albert Einstein :   1069460
2  Isaac Newton :   565211
3  Aristotle :   538371
4  Galileo Galilei :   410502
5  Srinivasa Ramanujan :   385192
6  René Descartes :   214691
7  Aryabhata :   176654
8  Archimedes :   165054
9  John von Neumann :   128040
10  Johannes Kepler :   110973
